In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from tqdm import tqdm

In [ ]:
from sklearn_extra.cluster import KMedoids
from scipy.stats import wasserstein_distance

# Import and Cleaning

In [ ]:
df = pd.read_csv('E:\Documents\Google Drive\Eskwelabs\Sprint 5 - Capstone\data\consolidated_csv_01SUAL_G01.csv')
df.head()

In [ ]:
# Define the columns containing price-quantity pairs and timestamps
price_columns = [f'PRICE{i}' for i in range(1, 12)]
quantity_columns = [f'QUANTITY{i}' for i in range(1, 12)]

# specify selected price-quantity columns
selected_columns = ['RUN_TIME'] + [price for price in price_columns] + [quantity for quantity in quantity_columns]

In [ ]:
pd.set_option('display.max_columns', None)

# Apply the selection to the filtered DataFrame
result_df = df[selected_columns]

#change Uppercase to lowercase
result_df.columns = result_df.columns.str.lower()

In [ ]:
# Convert 'run_time' to datetime with multiple formats
result_df['temp_run_time'] = pd.to_datetime(result_df['run_time'], errors='coerce', format='%m/%d/%Y %I:%M:%S %p').copy()

# Handle the remaining date-only format separately
mask_date_only = result_df['temp_run_time'].dt.time == pd.Timestamp('00:00:00').time()
result_df['run_time'] = result_df['temp_run_time'].where(mask_date_only, result_df['temp_run_time'].combine_first(pd.to_datetime(result_df['temp_run_time'], errors='coerce', format='%m/%d/%Y')))

# Drop the temporary column
result_df.drop(columns=['temp_run_time'], inplace=True)

# Print the result
result_df.head()

In [ ]:
starting_index = 6
downsampled_df = result_df.iloc[starting_index::12, :].copy()  # Use .copy() to create a copy of the DataFrame
downsampled_df.reset_index(drop=True, inplace=True)
downsampled_df

## FFill NaNs with last Price-Quantity bid

In [ ]:
# Forward fill from the first non-null value in each row for the price columns
downsampled_df.loc[:, 'price1':'price11'] = downsampled_df.loc[:, 'price1':'price11'].apply(lambda row: row.ffill(), axis=1)

# Forward fill from the first non-null value in each row for the quantity columns
downsampled_df.loc[:, 'quantity1':'quantity11'] = downsampled_df.loc[:, 'quantity1':'quantity11'].apply(lambda row: row.ffill(), axis=1)

downsampled_df.head()

## MinMaxScaler on Quantity values

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Extract only the quantity columns for normalization
quantity_columns = [f'quantity{i}' for i in range(1, 12)]

# Flatten the DataFrame and extract only quantity columns
flattened_quantities = downsampled_df[quantity_columns].values.flatten()

# Reshape the flattened quantities to a column vector
flattened_quantities = flattened_quantities.reshape(-1, 1)

# Use MinMaxScaler on the flattened quantities
scaler = MinMaxScaler()
scaled_quantities = scaler.fit_transform(flattened_quantities)

# Reshape the scaled quantities to match the original DataFrame shape
scaled_quantities = scaled_quantities.reshape(downsampled_df[quantity_columns].shape)

# Update the DataFrame with the scaled values
downsampled_df.loc[:, quantity_columns] = scaled_quantities

## Filter df for only 2022

In [ ]:
# Filter rows within the year 2022
df_2022 = downsampled_df[(downsampled_df['run_time'] >= '2022-01-01') & (downsampled_df['run_time'] < '2023-01-01')]

# Reset index
df_2022 = df_2022.reset_index(drop=True)
df_2022

## EDA

In [ ]:
df_2022.info()

In [ ]:
df_2022.describe()

## Check for NaNs

In [ ]:
missing_values = df_2022.isnull().sum()
print("Missing Values:")
print(missing_values)

## Visualize with Histograms

In [ ]:
# Extract prices from df_2022
prices_columns = [f'price{i}' for i in range(1, 12)]
prices = df_2022[prices_columns].values.flatten()

# Plot histogram
plt.hist(prices, bins=20, color='blue', alpha=0.7)
plt.xlabel('Prices')
plt.ylabel('Frequency')
plt.title('Distribution of Prices')
plt.show()

In [ ]:
# Extract quantities from df_2022
quantities_columns = [f'quantity{i}' for i in range(1, 12)]
quantities = df_2022[quantities_columns].values.flatten()

# Plot histogram
plt.hist(quantities, bins=20, color='green', alpha=0.7)
plt.xlabel('Quantities')
plt.ylabel('Frequency')
plt.title('Distribution of Quantities')
plt.show()

# K-medoids on subset of 2022 data

## Re-shape df into array

#### Select subset of df

In [ ]:
# Select only the first 100 entries from df_2022
df_subset = df_2022.head(100)

#### Test plot random bid curves from df_subset

In [ ]:
# Test Plot random indices from subset df
# Randomly select 10 indices
random_indices = df_subset.sample(n=10).index

# Extract prices and cumulative quantities for the randomly selected indices
selected_prices = df_subset.loc[random_indices, 'price1':'price11'].values
selected_cumulative_quantities = df_subset.loc[random_indices, 'quantity1':'quantity11'].values

# Plotting the selected step-wise bid curves
for i, idx in enumerate(random_indices):
    cumulative_quantity = selected_cumulative_quantities[i]
    price = selected_prices[i]
    plt.step(cumulative_quantity, price, where='pre', label=f'Curve {idx}')

plt.xlabel('Cumulative Quantity (MW)')
plt.ylabel('Price (Peso/Megawatt-hr)')
plt.title('Randomly Selected Step-wise Bid Curves from df_subset')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Display the subset of df_subset for the randomly selected indices
df_subset_random = df_subset.loc[random_indices]
df_subset_random.head()

#### Make (quantity,price) tuples

In [ ]:
# Get values from columns
prices = df_subset[['price1', 'price2', 'price3', 'price4', 'price5', 'price6', 'price7', 'price8', 'price9', 'price10', 'price11']].values
quantities = df_subset[['quantity1', 'quantity2', 'quantity3', 'quantity4', 'quantity5', 'quantity6', 'quantity7', 'quantity8', 'quantity9', 'quantity10', 'quantity11']].values

# Create list of lists containting (quantity,prices) tuples
def create_stepwise_curves(quantities, prices):
    step_curves = []
    for i in range(len(prices)):
        step_curves.append(list(zip(quantities[i], prices[i])))
    return step_curves

step_curves = create_stepwise_curves(quantities, prices)

In [ ]:
# Check length of step_curves
num_curves = len(step_curves)
length_of_first_curve = len(step_curves[0]) if num_curves > 0 else 0

print(f"Number of curves: {num_curves}")
print(f"Length of the first curve: {length_of_first_curve}")

#### Change step_curves into array

In [ ]:
# Change step_curves into 2D numpy array
step_curves_array = np.array(step_curves).reshape(len(step_curves), -1)

In [ ]:
print("Shape of step_curves_array:", step_curves_array.shape)

## Initialize functions

In [ ]:
from sklearn_extra.cluster import KMedoids
from scipy.stats import wasserstein_distance
from joblib import Parallel, delayed
import numpy as np

# Function to calculate Wasserstein distance
def wasserstein_dist(p1, p2):
    return wasserstein_distance(p1, p2)

# Function to perform K-medoids clustering
def kmedoids_clustering(data, k):
    kmedoids = KMedoids(n_clusters=k, metric=wasserstein_dist, random_state=42)
    kmedoids.fit(data)
    return kmedoids.labels_, kmedoids.cluster_centers_

# Function to calculate Separation Threshold
def calculate_separation_threshold(labels, centers, separation_threshold, p_ref):
    k = len(centers)
    s_ref = wasserstein_dist(centers[0], p_ref)  # Assuming P_ref is the first center
    s_th = separation_threshold * s_ref
    
    for i in range(k):
        distances_within_cluster = [
            wasserstein_dist(step_curves_array[j], centers[i]) 
            for j in range(len(labels)) if labels[j] == i
        ]
        rho_i = sum(1 for dist in distances_within_cluster if dist > s_th) / len(distances_within_cluster)

        if rho_i > tolerance_rate:
            return True

    return False

# Modify process_chunk to return labels along with result
def process_chunk(K, step_curves_array, separation_threshold, tolerance_rate):
    labels, centers = kmedoids_clustering(step_curves_array, K)
    print(f"Processed chunk for K={K}: Labels: {labels}, Centers Shape: {np.shape(centers)}")

    # Check for empty clusters
    empty_clusters = [i for i, cluster_size in enumerate(np.bincount(labels)) if cluster_size == 0]
    if empty_clusters:
        print(f"Warning: Empty clusters found for K={K}, Cluster indices: {empty_clusters}")

    # Print debugging information
    print("Cluster Centers:")
    print(centers)

    result = calculate_separation_threshold(labels, centers, separation_threshold, step_curves_array[0])

    return result, labels, centers


# Function to find optimal number of clusters
def find_optimal_clusters(step_curves_array, K_0, K_max, initial_chunk_size, iterations_within_chunk, separation_threshold, tolerance_rate):
    step_curves_array = step_curves_array.astype(np.float32)
    
    # Initialization
    K = K_0
    P_ref = step_curves_array[0]  # Using the first curve as a reference

    # Initialize the variable to track the optimal number of clusters
    optimal_K = K_0  

    # Store cluster centers and labels here
    centers_list = []  
    labels_list = []

    while K <= K_max:  
        print(f"Processing chunk for K={K}")

        result_found = False  # Flag to track whether a result was found in the inner loop
        # Inside your loop in find_optimal_clusters function
        for _ in range(iterations_within_chunk):
            chunk_size = min(initial_chunk_size, K_max - K)
            chunk_results = list(Parallel(n_jobs=-1)(delayed(process_chunk)(K + i, step_curves_array, separation_threshold, tolerance_rate) for i in range(chunk_size)))

            for result, labels, centers in chunk_results:
                if result:
                    optimal_K = K  # Update the optimal_K variable
                    centers_list.append(centers)
                    labels_list.append(labels)
                    result_found = True
                    break

            if result_found:
                break  # Break out of the inner loop if a result is found
                
        if result_found:
            break  # Break out of the outer loop if a result is found

        # Increment K after the chunk is processed
        K += chunk_size

    # Now optimal_K contains the optimal number of clusters, up to K_max
    print(f"Optimal number of clusters (K): {optimal_K}")
    
    # Return the labels_list
    return labels_list, centers_list

## Specify parameters

In [ ]:
# Parameters
K_0 = 2
K_max = 4  # Choose a smaller value for testing
separation_threshold = 0.01  # 1% as separation_threshold_ratio, increase if you think clusters should have larger dissimilarity
tolerance_rate = 0.05  # original was 0.1, increase if you want to be more strict about considering a cluster as separate

# Initial chunk size
initial_chunk_size = 5000
# Number of iterations within each chunk
iterations_within_chunk = 100

## Run

In [ ]:
%%time
# Find optimal clusters
found_labels_list, found_centers_list = find_optimal_clusters(step_curves_array, K_0, K_max, initial_chunk_size, iterations_within_chunk, separation_threshold, tolerance_rate)

## Results

### Apply cluster label to df

In [ ]:
# Assuming 'found_labels_list' is your list of labels after clustering
cluster_labels = found_labels_list[0]

# Create a copy of the subset to avoid SettingWithCopyWarning
df_subset_copy = df_subset.copy()

# Add 'cluster_no' column to df_subset_copy using .loc
df_subset_copy.loc[:, 'cluster_no'] = cluster_labels.copy()

# Now df_subset_copy has the 'cluster_no' column added without warnings
df_subset_copy


### Display cluster medoids

In [ ]:
# Assuming 'centers_list' is your list of cluster centers
final_centers = found_centers_list[-1]

# Assuming 'final_centers' is your cluster centers
num_features = final_centers.shape[1]
feature_columns = [f'quantity{i//2 + 1}' if i % 2 == 0 else f'price{i//2 + 1}' for i in range(num_features)]
centers_df = pd.DataFrame(final_centers, columns=feature_columns)
centers_df


### Visualize curves

In [ ]:
%%time
# Plotting parameters
thick_line_width = 4
thin_line_width = 1.0

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Get a color map
color_map = plt.get_cmap('tab10')

# Plotting cluster centers from 'centers_df' as thick lines
for index, center_row in centers_df.iterrows():
    cumulative_quantity, price = center_row.values.reshape(-1, 2).T
    plt.step(cumulative_quantity, price, where='pre', linewidth=thick_line_width, label=f'Cluster Medoid {index}', color=color_map(index / len(centers_df)))

# Plotting step-wise bid curves from 'df_subset_copy' as thinner lines
for index, row in df_subset_copy.iterrows():
    cluster_no = row['cluster_no']
    cumulative_quantity, price = row[['quantity1', 'price1', 'quantity2', 'price2', 'quantity3', 'price3', 'quantity4', 'price4', 'quantity5', 'price5', 'quantity6', 'price6', 'quantity7', 'price7', 'quantity8', 'price8', 'quantity9', 'price9', 'quantity10', 'price10', 'quantity11', 'price11']].values.reshape(-1, 2).T
    plt.step(cumulative_quantity, price, where='pre', linewidth=thin_line_width, linestyle='--', color=color_map(cluster_no / len(centers_df)))

# Plotting settings
plt.xlabel('Cumulative Quantity (MW)')
plt.ylabel('Price (Peso/Megawatt-hr)')
plt.title('Cluster Centers and Step-wise Bid Curves')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
%%time
# Plotting parameters
thick_line_width = 4
thin_line_width = 1.0

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Get a color map
color_map = plt.get_cmap('tab10')

# Plotting cluster centers from 'centers_df' as thick lines
for index, center_row in centers_df.iterrows():
    cumulative_quantity, price = center_row.values.reshape(-1, 2).T
    plt.step(cumulative_quantity, price, where='pre', linewidth=thick_line_width, label=f'Cluster Medoid {index}', color=color_map(index / len(centers_df)))

# Plotting step-wise bid curves from 'df_subset' as thinner lines
for index, row in df_subset.iterrows():
    cluster_no = row['cluster_no']
    cumulative_quantity, price = row[['quantity1', 'price1', 'quantity2', 'price2', 'quantity3', 'price3', 'quantity4', 'price4', 'quantity5', 'price5', 'quantity6', 'price6', 'quantity7', 'price7', 'quantity8', 'price8', 'quantity9', 'price9', 'quantity10', 'price10', 'quantity11', 'price11']].values.reshape(-1, 2).T
    plt.step(cumulative_quantity, price, where='pre', linewidth=thin_line_width, linestyle='--', color=color_map(cluster_no / len(centers_df)))

# Plotting settings
plt.xlabel('Cumulative Quantity (MW)')
plt.ylabel('Price (Peso/Megawatt-hr)')
plt.title('Cluster Centers and Step-wise Bid Curves')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
df_subset.info()

In [ ]:
# Assuming 'centers_df' is your DataFrame
for i in range(len(centers_df)):
    cumulative_quantity, price = zip(*centers_df.iloc[i].values.reshape(-1, 2))
    plt.step(cumulative_quantity, price, where='pre', label=f'Center {i + 1}')

plt.xlabel('Quantity (MW)')
plt.ylabel('Price (Peso/Megawatt-hr)')
plt.title('Bid Curves for Cluster Centers')
plt.legend()
plt.grid(True)
plt.show()


# K-medoids on entire 2022 df

In [ ]:
# Select only the first 100 entries from df_2022
df_subset = df_2022.head(7948)

# Get values from columns
prices = df_subset[['price1', 'price2', 'price3', 'price4', 'price5', 'price6', 'price7', 'price8', 'price9', 'price10', 'price11']].values
quantities = df_subset[['quantity1', 'quantity2', 'quantity3', 'quantity4', 'quantity5', 'quantity6', 'quantity7', 'quantity8', 'quantity9', 'quantity10', 'quantity11']].values

# Function to create step-wise bid curves
def create_stepwise_curves(prices, quantities):
    step_curves = []
    for i in range(len(prices)):
        step_curves.append(list(zip(quantities[i], prices[i])))
    return step_curves

step_curves = create_stepwise_curves(prices, quantities)


In [ ]:
num_curves = len(step_curves)
length_of_first_curve = len(step_curves[0]) if num_curves > 0 else 0

print(f"Number of curves: {num_curves}")
print(f"Length of the first curve: {length_of_first_curve}")

In [ ]:
# Test Plot random indices
# Randomly select 10 indices
random_indices = df_subset.sample(n=10).index

# Extract prices and cumulative quantities for the randomly selected indices
selected_prices = df_subset.loc[random_indices, 'price1':'price11'].values
selected_cumulative_quantities = df_subset.loc[random_indices, 'quantity1':'quantity11'].values

# Plotting the selected step-wise bid curves
for i, idx in enumerate(random_indices):
    cumulative_quantity = selected_cumulative_quantities[i]
    price = selected_prices[i]
    plt.step(cumulative_quantity, price, where='pre', label=f'Curve {idx}')

plt.xlabel('Cumulative Quantity (MW)')
plt.ylabel('Price (Peso/Megawatt-hr)')
plt.title('Randomly Selected Step-wise Bid Curves from df_subset')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Display the subset of df_subset for the randomly selected indices
df_subset_random = df_subset.loc[random_indices]
df_subset_random.head()

In [ ]:
# Change step_curves into numpy array
step_curves_array = np.array(step_curves).reshape(len(step_curves), -1)

In [ ]:
print("Shape of step_curves_array:", step_curves_array.shape)

## Run

In [ ]:
%%time
from sklearn_extra.cluster import KMedoids
from scipy.stats import wasserstein_distance
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

# Function to calculate Wasserstein distance
def wasserstein_dist(p1, p2):
    return wasserstein_distance(p1, p2)

# Function to perform K-medoids clustering
def kmedoids_clustering(data, k):
    kmedoids = KMedoids(n_clusters=k, metric=wasserstein_dist, random_state=42)
    kmedoids.fit(data)
    return kmedoids.labels_, kmedoids.cluster_centers_

# Function to calculate Separation Threshold
def separation_threshold_ratio(labels, centers, theta_0, p_ref):
    k = len(centers)
    s_ref = wasserstein_dist(centers[0], p_ref)  # Assuming P_ref is the first center
    s_th = theta_0 * s_ref
    
    for i in range(k):
        distances_within_cluster = [
            wasserstein_dist(step_curves_array[j], centers[i]) 
            for j in range(len(labels)) if labels[j] == i
        ]
        rho_i = sum(1 for dist in distances_within_cluster if dist > s_th) / len(distances_within_cluster)

        if rho_i > RHO_0:
            return True

    return False

# Function to process a chunk
def process_chunk(K):
    labels, centers = kmedoids_clustering(step_curves_array, K)
    print(f"Processed chunk for K={K}: Labels: {labels}, Centers Shape: {np.shape(centers)}")

    # Check for empty clusters
    empty_clusters = [i for i, cluster_size in enumerate(np.bincount(labels)) if cluster_size == 0]
    if empty_clusters:
        print(f"Warning: Empty clusters found for K={K}, Cluster indices: {empty_clusters}")

    # Add more debugging information or checks as needed

    return separation_threshold_ratio(labels, centers, THETA_0, P_ref), centers

In [ ]:
# Parameters
K_0 = 2
K_max = 7  # Choose a smaller value for testing
THETA_0 = 0.01  # 1% as THETA_0
RHO_0 = 0.075 #original was 0.1

# Initial chunk size
initial_chunk_size = 5000
# Number of iterations within each chunk
iterations_within_chunk = 500

# Reduce memory usage
step_curves_array = step_curves_array.astype(np.float32)

# Initialization
K = K_0
P_ref = step_curves_array[0]  # Using the first curve as a reference

# Initialize the variable to track the optimal number of clusters
optimal_K = K_0  

# Store cluster centers and labels here
centers_list = []  
labels_list = []

In [ ]:
%%time
with tqdm(total=K_max - K_0 + 1, desc="Iterating through K clusters") as pbar:
    while K <= K_max:  
        pbar.set_postfix({"Current K": K})

        # Dynamic chunk size based on remaining clusters
        remaining_clusters = K_max - K
        chunk_size = min(initial_chunk_size, remaining_clusters)

        print(f"Processing chunk for K={K}")

        result_found = False  # Flag to track whether a result was found in the inner loop

        for _ in range(iterations_within_chunk):
            # Use `list()` to force the progress bar to update in a Jupyter Notebook
            chunk_results = list(Parallel(n_jobs=-1)(delayed(process_chunk)(K + i) for i in range(chunk_size)))

            for result, centers in chunk_results:
                if result:
                    optimal_K = K  # Update the optimal_K variable
                    centers_list.append(centers)
                    labels_list.append(kmedoids_clustering(step_curves_array, K)[0])
                    result_found = True
                    break

            if result_found:
                break  # Break out of the inner loop if a result is found

            pbar.update(1)  

            # Print additional information every 100 iterations
            if K % 100 == 0:
                print(f"Current iteration: {K}, Result: {result}")

        if result_found:
            break  # Break out of the outer loop if a result is found

        # Increment K after the chunk is processed
        K += chunk_size

# Now optimal_K contains the optimal number of clusters, up to K_max
print(f"Optimal number of clusters (K): {optimal_K}")

In [ ]:
labels_list

In [ ]:
# Assuming 'labels_list' is your list of labels after K-means clustering
cluster_labels = labels_list[0]

# Add 'cluster_no' column to df_subset using .loc
df_subset.loc[:, 'cluster_no'] = cluster_labels
df_subset

In [ ]:
# Assuming 'centers_list' is your list of cluster centers
final_centers = centers_list[-1]

# Assuming 'final_centers' is your cluster centers
num_features = final_centers.shape[1]
feature_columns = [f'quantity{i//2 + 1}' if i % 2 == 0 else f'price{i//2 + 1}' for i in range(num_features)]
centers_df = pd.DataFrame(final_centers, columns=feature_columns)
centers_df


In [ ]:
df_subset.describe()

### Visualize curves

In [ ]:
%%time
# Plotting parameters
thick_line_width = 5
thin_line_width = 0.5

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Get a color map
color_map = plt.get_cmap('tab10')

# Plotting step-wise bid curves from 'df_subset' as thinner lines
for index, row in df_subset.iterrows():
    cluster_no = row['cluster_no']
    cumulative_quantity, price = row[['quantity1', 'price1', 'quantity2', 'price2', 'quantity3', 'price3', 'quantity4', 'price4', 'quantity5', 'price5', 'quantity6', 'price6', 'quantity7', 'price7', 'quantity8', 'price8', 'quantity9', 'price9', 'quantity10', 'price10', 'quantity11', 'price11']].values.reshape(-1, 2).T
    plt.step(cumulative_quantity, price, where='pre', linewidth=thin_line_width, linestyle='--', color=color_map(cluster_no / len(centers_df)))

# Plotting cluster centers from 'centers_df' as thick lines
for index, center_row in centers_df.iterrows():
    cumulative_quantity, price = center_row.values.reshape(-1, 2).T
    plt.step(cumulative_quantity, price, where='pre', linewidth=thick_line_width, label=f'Cluster Medoid {index}', color=color_map(index / len(centers_df)))

# Plotting settings
plt.xlabel('Cumulative Quantity (MW)')
plt.ylabel('Price (Peso/Megawatt-hr)')
plt.title('Cluster Centers and Step-wise Bid Curves')
plt.legend()
plt.grid(True)
plt.show()


# [WIP] K-medoids with both Price-Quantity Scaled

In [ ]:
# Select only the first 100 entries from df_2022
df_subset = df_2022.head(500)

In [ ]:
# Get values from columns
prices = df_subset[['price1', 'price2', 'price3', 'price4', 'price5', 'price6', 'price7', 'price8', 'price9', 'price10', 'price11']].values
quantities = df_subset[['quantity1', 'quantity2', 'quantity3', 'quantity4', 'quantity5', 'quantity6', 'quantity7', 'quantity8', 'quantity9', 'quantity10', 'quantity11']].values

## RobustScaler on Price values

In [ ]:
from sklearn.preprocessing import RobustScaler

# # Assuming prices is a DataFrame with columns price1 to price11
# prices_columns = [f'price{i}' for i in range(1, 12)]
# prices = df_subset[prices_columns]

# Apply Robust scaling to prices
scaler_prices = RobustScaler()
prices_scaled = scaler_prices.fit_transform(prices)
prices_scaled

In [ ]:

# statistics

prices_scaled_stats = {
    'mean': np.mean(prices_scaled, axis=0),
    'std': np.std(prices_scaled, axis=0),
    'min': np.min(prices_scaled, axis=0),
    '25%': np.percentile(prices_scaled, 25, axis=0),
    '50%': np.percentile(prices_scaled, 50, axis=0),
    '75%': np.percentile(prices_scaled, 75, axis=0),
    'max': np.max(prices_scaled, axis=0),
}

# Create a DataFrame for better visualization
prices_scaled_stats_df = pd.DataFrame(prices_scaled_stats, index=[f'Price_{i+1}' for i in range(prices_scaled.shape[1])])

# Display the statistics DataFrame
print(prices_scaled_stats_df)


In [ ]:
# Specify the desired range based on your data
desired_range = (-10, 10)

# Flatten the 2D array into a 1D array
scaled_prices_flat = prices_scaled.flatten()

# Plot histogram with specified range
plt.hist(scaled_prices_flat, bins=20, range=desired_range, color='blue', alpha=0.7)
plt.xlabel('Scaled Prices')
plt.ylabel('Frequency')
plt.title('Distribution of Scaled Prices')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'prices_scaled' is your array
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))

# Create a box plot to visualize the distribution
sns.boxplot(data=prices_scaled, orient="v", palette="Set2")

plt.title('Box Plot of Scaled Prices')
plt.ylabel('Scaled Prices')
plt.show()


In [ ]:
# Using prices_scaled
def create_stepwise_curves_scaled(prices, quantities):
    step_curves = []
    for i in range(len(prices)):
        step_curves.append(list(zip(quantities[i], prices[i])))
    return step_curves

# Create list of lists containting price,quantity tuples
step_curves = create_stepwise_curves_scaled(prices_scaled, quantities)
step_curves

In [ ]:
num_curves = len(step_curves)
length_of_first_curve = len(step_curves[0]) if num_curves > 0 else 0

print(f"Number of curves: {num_curves}")
print(f"Length of the first curve: {length_of_first_curve}")

In [ ]:
# Test Plot random indices
# Randomly select 10 indices
random_indices = df_subset.sample(n=10).index

# Extract prices and cumulative quantities for the randomly selected indices
selected_prices = df_subset.loc[random_indices, 'price1':'price11'].values
selected_cumulative_quantities = df_subset.loc[random_indices, 'quantity1':'quantity11'].values

# Plotting the selected step-wise bid curves
for i, idx in enumerate(random_indices):
    cumulative_quantity = selected_cumulative_quantities[i]
    price = selected_prices[i]
    plt.step(cumulative_quantity, price, where='pre', label=f'Curve {idx}')

plt.xlabel('Cumulative Quantity (MW)')
plt.ylabel('Price (Peso/Megawatt-hr)')
plt.title('Randomly Selected Step-wise Bid Curves from df_subset')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Display the subset of df_subset for the randomly selected indices
df_subset_random = df_subset.loc[random_indices]
# df_subset_random

In [ ]:
# Change step_curves into 2D numpy array 
step_curves_array = np.array(step_curves).reshape(len(step_curves), -1)
step_curves_array

In [ ]:
print("Shape of step_curves_array:", step_curves_array.shape)

## K-medoids Process

In [ ]:
from sklearn_extra.cluster import KMedoids
from scipy.stats import wasserstein_distance
from joblib import Parallel, delayed
import numpy as np

# Function to calculate Wasserstein distance
def wasserstein_dist(p1, p2):
    return wasserstein_distance(p1, p2)

# Function to perform K-medoids clustering
def kmedoids_clustering(data, k):
    kmedoids = KMedoids(n_clusters=k, metric=wasserstein_dist, random_state=42)
    kmedoids.fit(data)
    return kmedoids.labels_, kmedoids.cluster_centers_

# Function to calculate Separation Threshold
def calculate_separation_threshold(labels, centers, separation_threshold, p_ref):
    k = len(centers)
    s_ref = wasserstein_dist(centers[0], p_ref)  # Assuming P_ref is the first center
    s_th = separation_threshold * s_ref
    
    for i in range(k):
        distances_within_cluster = [
            wasserstein_dist(step_curves_array[j], centers[i]) 
            for j in range(len(labels)) if labels[j] == i
        ]
        rho_i = sum(1 for dist in distances_within_cluster if dist > s_th) / len(distances_within_cluster)

        if rho_i > tolerance_rate:
            return True

    return False

# Modify process_chunk to return labels along with result
def process_chunk(K, step_curves_array, separation_threshold, tolerance_rate):
    labels, centers = kmedoids_clustering(step_curves_array, K)
    print(f"Processed chunk for K={K}: Labels: {labels}, Centers Shape: {np.shape(centers)}")

    # Check for empty clusters
    empty_clusters = [i for i, cluster_size in enumerate(np.bincount(labels)) if cluster_size == 0]
    if empty_clusters:
        print(f"Warning: Empty clusters found for K={K}, Cluster indices: {empty_clusters}")

    # Print debugging information
    print("Cluster Centers:")
    print(centers)

    result = calculate_separation_threshold(labels, centers, separation_threshold, step_curves_array[0])

    return result, labels, centers


# Function to find optimal number of clusters
def find_optimal_clusters(step_curves_array, K_0, K_max, initial_chunk_size, iterations_within_chunk, separation_threshold, tolerance_rate):
    step_curves_array = step_curves_array.astype(np.float32)
    
    # Initialization
    K = K_0
    P_ref = step_curves_array[0]  # Using the first curve as a reference

    # Initialize the variable to track the optimal number of clusters
    optimal_K = K_0  

    # Store cluster centers and labels here
    centers_list = []  
    labels_list = []

    while K <= K_max:  
        print(f"Processing chunk for K={K}")

        result_found = False  # Flag to track whether a result was found in the inner loop
        # Inside your loop in find_optimal_clusters function
        for _ in range(iterations_within_chunk):
            chunk_size = min(initial_chunk_size, K_max - K)
            chunk_results = list(Parallel(n_jobs=-1)(delayed(process_chunk)(K + i, step_curves_array, separation_threshold, tolerance_rate) for i in range(chunk_size)))

            for result, labels, centers in chunk_results:
                if result:
                    optimal_K = K  # Update the optimal_K variable
                    centers_list.append(centers)
                    labels_list.append(labels)
                    result_found = True
                    break

            if result_found:
                break  # Break out of the inner loop if a result is found
                
        if result_found:
            break  # Break out of the outer loop if a result is found

        # Increment K after the chunk is processed
        K += chunk_size

    # Now optimal_K contains the optimal number of clusters, up to K_max
    print(f"Optimal number of clusters (K): {optimal_K}")
    
    # Return the labels_list
    return labels_list, centers_list

In [ ]:
%%time
# Parameters
K_0 = 2
K_max = 4  # Choose a smaller value for testing
separation_threshold = 0.01  # 1% as separation_threshold_ratio, increase if you think clusters should have larger dissimilarity
tolerance_rate = 0.05  # original was 0.1, increase if you want to be more strict about considering a cluster as separate

# Initial chunk size
initial_chunk_size = 8000
# Number of iterations within each chunk
iterations_within_chunk = 500

# Find optimal clusters
found_labels_list, found_centers_list = find_optimal_clusters(step_curves_array, K_0, K_max, initial_chunk_size, iterations_within_chunk, separation_threshold, tolerance_rate)

In [ ]:
# Assuming 'found_labels_list' is your list of labels after clustering
cluster_labels = found_labels_list[0]

# Add 'cluster_no' column to df_subset using .loc
df_subset.loc[:, 'cluster_no'] = cluster_labels.copy()  # Use .copy() to avoid the warning
df_subset

## Reconstitute centers df

In [ ]:
found_centers_list

In [ ]:
import pandas as pd

# Assuming 'step_curves_array' is your original data for scaling
num_features = len(step_curves[0][0])  # Assuming each step has 'num_features' features
price_indices = range(1, num_features * len(step_curves[0]), num_features)

# Calculate min and max prices
min_prices = step_curves_array[:, price_indices].min(axis=0)
max_prices = step_curves_array[:, price_indices].max(axis=0)
max_prices

In [ ]:
# Function to inverse scale prices
def inverse_scale_prices(scaled_prices, min_prices, max_prices):
    return (scaled_prices * (max_prices - min_prices)) + min_prices

# Create an empty DataFrame for centers_df
centers_df = pd.DataFrame()

# Iterate over the clusters and inverse scale prices
for i, cluster_center in enumerate(found_centers_list):
    # Assuming 'found_centers_list' is a list of numpy arrays for each cluster center
    center_df = pd.DataFrame(cluster_center, columns=[f'quantity{i+1}', f'price{i+1}'])

    # Inverse scale prices for the current cluster
    center_df[f'price{i+1}'] = inverse_scale_prices(center_df[f'price{i+1}'], min_prices, max_prices)

    # Concatenate to the overall centers_df
    centers_df = pd.concat([centers_df, center_df], axis=1)

# Display the final DataFrame
centers_df